In [1]:
%%info

In [1]:
#importing necessary libaries
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark import SQLContext
from itertools import islice
from pyspark.sql.functions import col

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1603723084707_0008,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import json

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#creating the context
sqlContext = SQLContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
original_books = sc.textFile("s3n://analytics-50043/data/meta_Kindle_Store_original.json")\
                    .map(lambda line: json.loads(line)).filter(lambda line: line['title'] != '')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:

import re
import html

def clean_string(string):
    string = re.sub('<[^<]+?>', '', string)
    return html.unescape(string).strip()

def clean_rank(rank_string):
    if not isinstance(rank_string, str):
        return rank_string
    return clean_string(rank_string.split(' ')[0].replace(',', ''))

def clean_author(brand_string):
    if not isinstance(brand_string, str):
        return ""
    if (len(brand_string) > 20 and brand_string[:14].lower() == "visit amazon's" and brand_string[-4:].lower() == 'page'):
        return clean_string(brand_string[15: -5])
    return clean_string(brand_string)
    
def merge_two_list(list1, list2):
    list1.extend(x for x in list2 if x not in list1)
    return list1

def get_single_cate(cate_list):
    return '' if len(cate_list) < 3 else clean_string(cate_list[2])

def clean_title(string):
    # not seperated
    if len(string) >= 250 and len(string.split(',')) >= 4:
        string = string.split(',')[2]
    return clean_string(string.replace(' - Kindle edition', ''))

print(clean_title('By Force of Instinct: A Pride &amp; Prejudice Variation - Kindle edition'))

print(clean_rank('1,716,849 Paid in Kindle Store ('))
print(clean_author("Visit Amazon's Paul A. Craig Page"))
(print(merge_two_list(['1','3','5'],['5','3', 7,8,'8'])))
print(clean_string('<html>i am a title &amp; I am mom&#39;s good boy</html>'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

By Force of Instinct: A Pride & Prejudice Variation
1716849
Paul A. Craig
['1', '3', '5', 7, 8, '8']
i am a title & I am mom's good boy

In [21]:
books = original_books.map(lambda line: {'ASIN': line['asin'], 'category': get_single_cate(line['category']), 'title': clean_title(line['title']), 
                                         'author': clean_author(line['brand']),
                                       'image': line['image'], 'description': line['description'], 'rank': clean_rank(line['rank']),
                                      'recommendation': line['also_view'], 'price': line['price']})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
books.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

488910

In [10]:
books.first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ASIN': 'B00128EFK0', 'category': 'Religion & Spirituality', 'title': "Approaching the Holy-A Season in God's House", 'author': 'Dennis G. Crump', 'image': [], 'description': [], 'rank': '4646191', 'recommendation': [], 'price': ''}

In [11]:
books_list = books.take(30)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
for i in books_list:
    print(i)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ASIN': 'B00128EFK0', 'category': 'Religion & Spirituality', 'title': "Approaching the Holy-A Season in God's House", 'author': 'Dennis G. Crump', 'image': [], 'description': [], 'rank': '4646191', 'recommendation': [], 'price': ''}
{'ASIN': 'B00129LKMK', 'category': 'Arts & Photography', 'title': "Bellini's I PURITANI Opera Journeys Mini Guide (Opera Journeys Mini Guide Series) eBook", 'author': 'Burton D. Fisher', 'image': [], 'description': [], 'rank': '3489361', 'recommendation': [], 'price': ''}
{'ASIN': 'B0012AXRXO', 'category': 'Mystery, Thriller & Suspense', 'title': 'Snowman', 'author': 'Dave Saari', 'image': [], 'description': [], 'rank': '2283726', 'recommendation': [], 'price': ''}
{'ASIN': 'B0012BKPEW', 'category': 'Literature & Fiction', 'title': 'Crumbs and Crutons', 'author': 'David Greer', 'image': [], 'description': [], 'rank': '4741892', 'recommendation': [], 'price': ''}
{'ASIN': 'B0012BS7UQ', 'category': 'Religion & Spirituality', 'title': 'Fast Track to Victory, 

In [13]:
with_price = books.filter(lambda line: line['price'] != '').count()
with_recommendation = books.filter(lambda line: line['recommendation'] != []).count()
with_image = books.filter(lambda line: line['image'] != []).count()
with_desc = books.filter(lambda line: line['description'] != []).count()
print("with price number of books : {}".format(with_price))
print("with recommendation number of books : {}".format(with_recommendation))
print("with image number of books : {}".format(with_image))
print("with desc number of books : {}".format(with_desc))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

with price number of books : 6
with recommendation number of books : 130578
with image number of books : 0
with desc number of books : 0

In [14]:
title_author = books.map(lambda line: (line['title'], line['author']))
collected = title_author.take(300)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
for i in collected:
    print(i)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

("Approaching the Holy-A Season in God's House", 'Dennis G. Crump')
("Bellini's I PURITANI Opera Journeys Mini Guide (Opera Journeys Mini Guide Series) eBook", 'Burton D. Fisher')
('Snowman', 'Dave Saari')
('Crumbs and Crutons', 'David Greer')
('Fast Track to Victory, A Christian Guidebook', 'Cheryl Rogers')
('To Roma! To Roma! (Italian Art Theft Adventures Book 1)', 'John Galavan')
('How to Study Smarter Not Harder', 'Dr. Charlotte Garman')
('Building Self-Esteem in Childhood & Adolescence: A Guide for Parents, Teachers, Coaches & Youth Workers (Child Psychology & Mental Health Book 5)', 'Amy R. Vigilante')
('Hunting Gear You Can Build and Much More! eBook', 'Vernon A. Blackledge')
('Anxiety Disorders in Childhood & Adolescence', 'Cyma J. Siegel')
('Brother One Cell', 'Cullen Thomas')
('Longarm 352: Longarm and the Pine Box Payoff', 'Tabor Evans')
('Dictionnaire Argot-francais (French Edition) eBook', 'Napoleon Hayard')
('Juvenile Sex Offenders', 'Cynthia Godbey')
('Bang Crunch (Vinta

In [23]:
from pyspark.sql.types import StructType,StructField,StringType
fields =['asin','category','title','author','rank','recommendation']

schema = StructType([
StructField(field,StringType(),True) for field in fields
])


#rdd = sc.parallelize(js)
jsDF = spark.createDataFrame(books,schema)

jsDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+--------------------+--------------------+-------+--------------------+
|asin|            category|               title|              author|   rank|      recommendation|
+----+--------------------+--------------------+--------------------+-------+--------------------+
|null|Religion & Spirit...|Approaching the H...|     Dennis G. Crump|4646191|                  []|
|null|  Arts & Photography|Bellini's I PURIT...|    Burton D. Fisher|3489361|                  []|
|null|Mystery, Thriller...|             Snowman|          Dave Saari|2283726|                  []|
|null|Literature & Fiction|  Crumbs and Crutons|         David Greer|4741892|                  []|
|null|Religion & Spirit...|Fast Track to Vic...|       Cheryl Rogers|3182718|                  []|
|null|  Arts & Photography|To Roma! To Roma!...|        John Galavan|1853702|                  []|
|null|   Children's eBooks|How to Study Smar...|Dr. Charlotte Garman| 843165|        [B00I3QS1XQ]|
|null|Pare

In [25]:
categories = jsDF.groupBy('category').count()
categories.show(150, False)
categories.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------+------+
|category                                   |count |
+-------------------------------------------+------+
|Lesbian, Gay, Bisexual & Transgender eBooks|26    |
|Sports                                     |13    |
|Parenting & Relationships                  |4919  |
|Politics & Social Sciences                 |8702  |
|Biographies & Memoirs                      |11696 |
|Engineering & Transportation               |2073  |
|Romance                                    |34267 |
|Cookbooks, Food & Wine                     |10784 |
|45 minutes (22-32 pages)                   |559   |
|Nonfiction                                 |1829  |
|30 minutes (12-21 pages)                   |432   |
|Education & Teaching                       |8649  |
|History                                    |13384 |
|Kindle Keyboard                            |6     |
|Self-Help                                  |4460  |
|Industry Focus                             |1

In [26]:
authors = jsDF.select('author').distinct().count()
print(authors)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

251498